# numba

numba es un compilador de Python a LLVM, que funciona para un subconjunto de la funcionalidad del lenguaje orientado a código científico en el que predomina la manipulación de arrays. Este notebook está basado en nuestro artículo [cómo acelerar código Python con numba](http://pybonacci.org/2015/03/13/como-acelerar-tu-codigo-python-con-numba/)

En la web puedes ver la [evolución del rendimiento](http://numba.pydata.org/numba-benchmark/) de algunos benchmarks gracias a [asv](http://github.com/spacetelescope/asv/).

## ¿Cómo funciona?

http://numba.pydata.org/numba-doc/0.18.2/developer/architecture.html

## Entendiendo numba: el modo *nopython*

Como podemos leer en la documentación, [numba tiene dos modos de funcionamiento básicos](http://numba.pydata.org/numba-doc/0.17.0/user/jit.html#nopython): el modo *object* y el modo *nopython*.

* El modo *object* genera código que gestiona todas las variables como objetos de Python y utiliza la API C de Python para operar con ellas. En general en este modo **no habrá ganancias de rendimiento** (e incluso puede ir más lento), con lo cual mi recomendación personal es directamente no utilizarlo. Hay casos en los que numba puede detectar los bucles y optimizarlos individualmente (*loop-jitting*), pero no le voy a prestar mucha atención a esto.
* El modo *nopython* **genera código independiente de la API C de Python**. Esto tiene la desventaja de que no podemos usar todas las características del lenguaje, **pero tiene un efecto significativo en el rendimiento**. Otra de las restricciones es que **no se puede reservar memoria para objetos nuevos**.

Por defecto numba usa el modo *nopython* siempre que puede, y si no pasa a modo *object*. Nosotros vamos a **forzar el modo nopython** (o «modo estricto» como me gusta llamarlo) porque es la única forma de aprovechar el potencial de numba.

## Ámbito de aplicación

El problema del modo *nopython* es que los mensajes de error son totalmente inservibles en la mayoría de los casos, así que antes de lanzarnos a compilar funciones con numba conviene hacer un repaso de qué no podemos hacer para anticipar la mejor forma de programar nuestro código. Podéis consultar en la documentación [el subconjunto de Python soportado por numba](http://numba.pydata.org/numba-doc/0.17.0/reference/pysupported.html) en modo *nopython*, y ya os aviso que, al menos de momento, no tenemos [*list comprehensions*](https://github.com/numba/numba/issues/504), [generadores](https://github.com/numba/numba/issues/984) ni algunas cosas más. Permitidme que resalte una frase sacada de la página principal de numba:

> "*With a few annotations, **array-oriented and math-heavy Python code** can be just-in-time compiled to native machine instructions, similar in performance to C, C++ and Fortran*". [Énfasis mío]

Siento decepcionar a la audiencia pero *numba no acelerará todo el código Python* que le echemos: está enfocado a operaciones matemáticas con arrays. Aclarado este punto, vamos a ponernos manos a la obra con un ejemplo aplicado :)

### Solución de Navier de una placa plana

Para mi proyecto fin de carrera me encontré con la necesidad de calcular la deflexión de una placa rectangular, simplemente apoyada en sus cuatro bordes (es decir, los bordes pueden girar: no están empotrados) sometida a una carga transversal. Este problema tiene solución analítica conocida desde hace tiempo, hallada por Navier:

$$w(x,y) = \sum_{m=1}^\infty \sum_{n=1}^\infty \frac{a_{mn}}{\pi^4 D}\,\left(\frac{m^2}{a^2}+\frac{n^2}{b^2}\right)^{-2}\,\sin\frac{m \pi x}{a}\sin\frac{n \pi y}{b}$$

siendo $a_{mn}$ los coeficientes de Fourier de la carga aplicada. Como veis, para cada punto $(x, y)$ hay que hacer una doble suma en serie; si encima queremos evaluar esto en un `meshgrid`, necesitamos **un cuádruple bucle**. Ya se anticipa que por muy hábiles que estemos, a Python le va a costar.

La clave estuvo, una vez más, en usar numba para optimizar los bucles. En GitHub tenéis [el código completo](https://gist.github.com/Juanlu001/cf19b1c16caf618860fb), pero la parte importante es esta:

¿Qué pasa si intentamos aplicar `numba.jit` a la función tal cual?

In [5]:
import numpy as np
from numpy import sin, pi
import numba

def plate_displacement_py(xx, yy, ww, a, b, P, xi, eta, D, max_i, max_j, max_m, max_n):
    for mm in range(1, max_m):
        for nn in range(1, max_n):
            for ii in range(max_i):
                for jj in range(max_j):
                    a_mn = 4 * P * sin(mm * pi * xi / a) * sin(nn * pi * eta / b) / (a * b)
                    ww[ii][jj] += (a_mn / (mm**2 / a**2 + nn**2 / b**2)**2
                                   * sin(mm * pi * xx[ii][jj] / a)
                                   * sin(nn * pi * yy[ii][jj] / b)
                                   / (pi**4 * D))

In [6]:
plate_displacement_nb = numba.jit(plate_displacement_py)

In [7]:
# Plate geometry
a = 1.0  # m
b = 1.0  # m
h = 50e-3  # m

# Material properties
E = 69e9  # Pa
nu = 0.35

# Series terms
max_m = 16
max_n = 16

# Computation points
# NOTE: With an odd number of points the center of the place is included in
# the grid
NUM_POINTS = 101
max_i = NUM_POINTS
max_j = NUM_POINTS

# Load
P = 10e3  # N
xi = a / 2
eta = a / 2

# Flexural rigidity
D = h**3 * E / (12 * (1 - nu**2))

# ---

# Set up domain
x = np.linspace(0, a, num=NUM_POINTS)
y = np.linspace(0, b, num=NUM_POINTS)
xx, yy = np.meshgrid(x, y)


In [9]:
# Compute displacement field
ww = np.zeros_like(xx)
plate_displacement_nb(xx, yy, ww, a, b, P, xi, eta, D, max_i, max_j, max_m, max_n)

# Print maximum displacement
w_max = np.abs(ww).max()
print("Maximum displacement = %14.12f mm" % (w_max * 1e3))
print("alpha = %7.5f" % (w_max / (P * a**2 / D)))
print("alpha * P a^2 / D = %6.4f mm" % (0.01160 * P * a**2 / D * 1e3))

Maximum displacement = 0.141317840389 mm
alpha = 0.01158
alpha * P a^2 / D = 0.1416 mm


In [11]:
%timeit plate_displacement_nb(xx, yy, ww, a, b, P, xi, eta, D, max_i, max_j, max_m, max_n)

1 loops, best of 3: 1.13 s per loop


**No hay mejoras significativas en el rendimiento**. Esto es en parte por todos los problemas que hemos dicho antes. Si probamos desde la versión de NumPy las cosas no mejoran demasiado:

In [13]:
def plate_displacement_np(xx, yy, ww, a, b, P, xi, eta, D, max_m, max_n):
    for mm in range(1, max_m):
        for nn in range(1, max_n):
            a_mn = 4 * P * sin(mm * pi * xi / a) * sin(nn * pi * eta / b) / (a * b)
            ww += (a_mn / (mm**2 / a**2 + nn**2 / b**2)**2
                   * sin(mm * pi * xx / a)
                   * sin(nn * pi * yy / b)
                   / (pi**4 * D))

plate_displacement_nb2 = numba.jit(plate_displacement_np)

In [15]:
# Compute displacement field
ww = np.zeros_like(xx)
plate_displacement_nb2(xx, yy, ww, a, b, P, xi, eta, D, max_m, max_n)

# Print maximum displacement
w_max = np.abs(ww).max()
print("Maximum displacement = %14.12f mm" % (w_max * 1e3))
print("alpha = %7.5f" % (w_max / (P * a**2 / D)))
print("alpha * P a^2 / D = %6.4f mm" % (0.01160 * P * a**2 / D * 1e3))

Maximum displacement = 0.141317840389 mm
alpha = 0.01158
alpha * P a^2 / D = 0.1416 mm


In [16]:
%timeit plate_displacement_nb2(xx, yy, ww, a, b, P, xi, eta, D, max_m, max_n)

1 loops, best of 3: 199 ms per loop


**El rendimiento es igual al de NumPy, sin más**. Podemos hacerlo un poco mejor, pero primero hay que dar un paso atrás.

## Final

In [25]:
@numba.jit(nopython=True)
def a_mn_point(P, a, b, xi, eta, mm, nn):
    """Navier series coefficient for concentrated load.

    """
    return 4 * P * sin(mm * pi * xi / a) * sin(nn * pi * eta / b) / (a * b)
 
 
@numba.jit(nopython=True)
def plate_displacement(xx, yy, ww, a, b, P, xi, eta, D, max_i, max_j, max_m, max_n):
    for mm in range(1, max_m):
        for nn in range(1, max_n):
            for ii in range(max_i):
                for jj in range(max_j):
                    a_mn = a_mn_point(P, a, b, xi, eta, mm, nn)
                    ww[ii, jj] += (a_mn / (mm**2 / a**2 + nn**2 / b**2)**2
                                   * sin(mm * pi * xx[ii, jj] / a)
                                   * sin(nn * pi * yy[ii, jj] / b)
                                   / (pi**4 * D)) 

In [26]:
# Compute displacement field
ww = np.zeros_like(xx)
plate_displacement(xx, yy, ww, a, b, P, xi, eta, D, max_i, max_j, max_m, max_n)

# Print maximum displacement
w_max = np.abs(ww).max()
print("Maximum displacement = %14.12f mm" % (w_max * 1e3))
print("alpha = %7.5f" % (w_max / (P * a**2 / D)))
print("alpha * P a^2 / D = %6.4f mm" % (0.01160 * P * a**2 / D * 1e3))

Maximum displacement = 0.141317840389 mm
alpha = 0.01158
alpha * P a^2 / D = 0.1416 mm


In [28]:
%timeit plate_displacement(xx, yy, ww, a, b, P, xi, eta, D, max_i, max_j, max_m, max_n)

1 loops, best of 3: 1.16 s per loop


<div class="alert alert-danger">**ALGO PASA**</div>

Podéis comprobar vosotros mismos que las diferencias de rendimiento en este caso son brutales. *Y solo hemos añadido una línea a cada función*.

## Conclusiones

numba aún no es una herramienta estable, pero está rápidamente alcanzando un grado de madurez suficiente para optimizar código orientado a operar con arrays. Gracias a conda es trivial de instalar y los resultados respecto a soluciones más maduras como Cython son aplastantes, tanto en velocidad de ejecución como en la complejidad del código resultante.

De momento yo me quedo con numba, ¿y tú? ;)